In [2]:
import numpy as np 
import networkx as nx
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
import random as rand
from collections import Counter

In [21]:
def generate_minimal_graph(n_nodes):
    g = nx.Graph()
    g.add_edges_from([(1,2), (2,3), (3,1)])
    for i in range(4, n_nodes+1):
        edges = list(g.edges)
        chosen_edge = rand.choice(edges)
        g.add_edge(chosen_edge[0], i)
        g.add_edge(chosen_edge[1], i)
    return g

In [32]:
def maki_thompson_rumour_model(graph: nx.Graph, gamma: float, alpha: float, time_limit: int, number_infected=1, all_time_values=False, starting_nodes=[], outputDictQ=True):

    #Clear status of the nodes of the given graph
    #"status" tells the type of node (I - Ignorant, S - Spreader, R - Stifler)
    #"visit" tells if the node should be visited in a cycle or not

    node_status={key: "I" for key in graph.nodes}
    node_visit={key: False for key in graph.nodes}
    nx.set_node_attributes(graph, node_status, "status")
    nx.set_node_attributes(graph, node_visit, "visit")

    #Obtain a random sample of size "number_infected" out of the nodes of the given graph
    if not(starting_nodes):
        starting_nodes = rand.sample(list(graph.nodes), number_infected)

    for node in starting_nodes:
        graph.nodes[node]["status"]="S"
        graph.nodes[node]["visit"]=True

        #All neighbours of spreaders should be visited in the cycle for possible changes
        for neighbor in graph.neighbors(node):
            graph.nodes[neighbor]["visit"]=True

    t=0

    #Counter for every status type, for each time value "t"
    counter=Counter(nx.get_node_attributes(graph, "status").values())
    if outputDictQ:
        status_count = [{"I": counter["I"], "S": counter["S"], "R": counter["R"]}]
    else:
        status_count= [[counter["I"], counter["S"], counter["R"]]]

    #Average number of messages sent per node, during the spreading process (If one node transforms to another type in the first time it gets transformed, should the other nodes still contribute
    # to the load?)
    load=0
    #Checks if there is still nodes worth visiting in the next iteration
    continue_visitingQ=True
    while t < time_limit and continue_visitingQ:
        #Only apply node_changes at the end of the cycle, the changes have to be done all at the same time.
        node_changes=[]
        for node in graph.nodes(data=True):
            if node[1]["visit"]==True:
                #Counting the amount of each type of nodes in the neighbourhood of the visited node
                neighbours_status_count={"I":0, "S":0, "R":0}
                for neighbor in graph.neighbors(node[0]):
                    neighbours_status_count[graph.nodes[neighbor]["status"]]+=1

                #If this value is less than the probability of transforming then the status of the node changes
                transform_value = rand.uniform(0,1)

                #Transformation of the node status
                if node[1]["status"]=="I":
                    #(1-gamma)**neighbours_status_count["S"] gives the probability of the event that no spreader transmits the rumour. We only need one of the spreaders to pass the rumour to transform the ignorant into spreader
                    transform_threshold = 1-(1-gamma)**neighbours_status_count["S"]
                    if transform_value <= transform_threshold:
                        node_changes.append((node[0], "S", True))
                        for neighbor in graph.neighbors(node[0]):
                            if graph.nodes[neighbor]["status"]!="R":
                                node_changes.append((neighbor, graph.nodes[neighbor]["status"], True))
                    load+=neighbours_status_count["S"]
                elif node[1]["status"]=="S":
                     #Whenever we are transforming a spreader into a stiffler the stifflers and the spreaders contribute the same to the probability of transforming the spreader
                    transform_threshold = 1-(1-alpha)**(neighbours_status_count["S"] + neighbours_status_count["R"])
                    if transform_value <= transform_threshold:
                        node_changes.append((node[0], "R", False))

                    load+=neighbours_status_count["S"]+neighbours_status_count["R"]
        
        #Applying all changes to the graph
        for node_change in node_changes:
            graph.nodes[node_change[0]]["status"]=node_change[1]
            graph.nodes[node_change[0]]["visit"]=node_change[2]
        
        counter=Counter(nx.get_node_attributes(graph, "status").values())

        if outputDictQ:
            status_count.append({"I": counter["I"], "S": counter["S"], "R": counter["R"]})
        else:
            status_count.append([counter["I"], counter["S"], counter["R"]])

        if counter["S"]==0:
            if not(all_time_values):
                continue_visitingQ=False
            else:
                for i in range(time_limit-t-2):
                    if outputDictQ:
                        status_count.append({"I": counter["I"], "S": counter["S"], "R": counter["R"]})
                    else:
                        status_count.append([counter["I"], counter["S"], counter["R"]])
                continue_visitingQ=False
        t+=1
    load=load/len(graph)
    return status_count, load, [i[1] for i in list(graph.degree())], list(nx.get_node_attributes(graph, "status").values())

In [ ]:
def plot_3_line_chart(x, y1, y2, y3, title="", x_title="", y_title=""):
    fig = go.Figure()

    # Add the first line
    fig.add_trace(go.Scatter(x=x, y=y1, mode='lines', name='Ignorant'))

    # Add the second line
    fig.add_trace(go.Scatter(x=x, y=y2, mode='lines', name='Spreader'))

    # Add the second line
    fig.add_trace(go.Scatter(x=x, y=y3, mode='lines', name='Stifler'))

    fig.update_xaxes(title=x_title)
    fig.update_yaxes(title=y_title)

    fig.update_layout(
        title=title,
        template='plotly',
        plot_bgcolor='white',
        width=800,
        height=400
    )

    fig.update_xaxes(
        mirror=False,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey'
    )
    fig.update_yaxes(
        mirror=False,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey'
    )

    return fig

def plot_stacked_area_chart(x, y1, y2, y3, title="", x_title="", y_title=""):
    fig = go.Figure()

    # Add the first line
    fig.add_trace(go.Scatter(x=x, y=y1, mode='lines', fill='tozeroy', name='Ignorant', stackgroup='one',
    groupnorm='percent'))

    # Add the second line
    fig.add_trace(go.Scatter(x=x, y=y2, mode='lines', fill='tonexty', name='Spreader', stackgroup='one'))

    # Add the second line
    fig.add_trace(go.Scatter(x=x, y=y3, mode='lines', fill='tonexty', name='Stifler', stackgroup='one'))

    fig.update_xaxes(title=x_title)
    fig.update_yaxes(title=y_title)

    fig.update_layout(
        title=title,
        template='plotly',
        plot_bgcolor='white',
        barmode='stack',
        width=800,
        height=400
    )

    fig.update_xaxes(
        mirror=False,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey'
    )
    fig.update_yaxes(
        mirror=False,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey'
    )

    return fig

In [26]:
ba_graph = nx.barabasi_albert_graph(n=10000,m=2)
ws_graph = nx.watts_strogatz_graph(n=10000, k=4, p=0.01)


In [46]:
simulation, load, degrees, status = maki_thompson_rumour_model(ba_graph, gamma=1, alpha=0.1, time_limit = 2000, number_infected=1, all_time_values=True)

In [47]:
print(len(simulation))
print(simulation[-1])
print(degrees)
print(status)
print(load)

2000
{'I': 0, 'S': 0, 'R': 10000}
[73, 2, 182, 227, 43, 71, 128, 164, 93, 73, 152, 90, 157, 24, 18, 143, 72, 98, 21, 56, 19, 66, 34, 73, 11, 36, 28, 22, 11, 45, 15, 42, 11, 43, 7, 22, 37, 38, 8, 12, 58, 15, 24, 25, 52, 33, 20, 9, 16, 46, 22, 22, 30, 8, 60, 49, 7, 16, 17, 21, 33, 33, 8, 10, 22, 28, 26, 26, 18, 33, 41, 32, 40, 31, 22, 18, 12, 10, 26, 41, 32, 21, 4, 20, 8, 6, 43, 18, 37, 39, 14, 2, 18, 4, 3, 5, 30, 13, 14, 15, 26, 6, 26, 7, 4, 30, 16, 58, 36, 24, 10, 35, 29, 31, 19, 11, 21, 12, 20, 17, 53, 21, 20, 14, 7, 22, 13, 24, 25, 16, 19, 4, 4, 3, 7, 11, 14, 11, 6, 10, 8, 25, 5, 7, 16, 13, 5, 24, 16, 8, 7, 25, 21, 9, 8, 9, 31, 8, 23, 26, 17, 3, 27, 21, 6, 20, 24, 18, 9, 18, 13, 19, 40, 8, 16, 21, 9, 31, 11, 13, 20, 28, 3, 7, 9, 8, 3, 29, 14, 18, 3, 22, 18, 10, 7, 7, 7, 37, 26, 14, 31, 10, 8, 16, 9, 11, 12, 10, 5, 18, 19, 34, 15, 4, 20, 7, 14, 18, 18, 8, 35, 10, 5, 28, 6, 5, 15, 14, 29, 10, 4, 10, 7, 3, 3, 33, 9, 6, 7, 14, 8, 17, 9, 9, 6, 11, 6, 4, 8, 30, 26, 4, 3, 4, 6, 5, 13, 6, 8,

In [ ]:
sim_records = []
for i, status in enumerate(simulation):
    sim_records.extend([(status["I"], status["S"], status["R"])])

sim_df = pd.DataFrame.from_records(sim_records, columns=["I", "S", "R"])

In [ ]:
plot_3_line_chart(x=sim_df.index, y1=sim_df["I"], y2=sim_df["S"], y3=sim_df["R"])

In [ ]:
plot_stacked_area_chart(x=sim_df.index, y1=sim_df["I"], y2=sim_df["S"], y3=sim_df["R"])

In [49]:
ba_results=[]
ws_results=[]
mm_results=[]
ba_degrees=[]
ws_degrees=[]
mm_degrees=[]
ba_mean_loads=[]
ws_mean_loads=[]
mm_mean_loads=[]

for alpha in np.linspace(0,1,20):
    ba_alpha_results=[]
    ws_alpha_results=[]
    mm_alpha_results=[]
    ba_alpha_degrees=[]
    ws_alpha_degrees=[]
    mm_alpha_degrees=[]
    ba__alpha_mean_loads=[]
    ws_alpha_mean_loads=[]
    mm_alpha_mean_loads=[]
    ba__results=[]
    ws_alpha_results=[]
    mm_alpha_results=[]
    
    for j in range(10):
        ba_graph = nx.barabasi_albert_graph(n=10,m=2)
        ws_graph = nx.watts_strogatz_graph(n=10, k=4, p=0.01)
        mm_graph = generate_minimal_graph(n_nodes=10) 
        ba_sim, ba_load, ba_j_degree, ba_status = maki_thompson_rumour_model(ba_graph, gamma=0.01, alpha=alpha, time_limit=20, number_infected=1, all_time_values=True, outputDictQ=False)
        ws_sim, ws_load, ws_j_degree, ws_status = maki_thompson_rumour_model(ws_graph, gamma=0.01, alpha=alpha, time_limit=20, number_infected=1, all_time_values=True, outputDictQ=False)
        mm_sim, ws_load, ws_j_degree, ws_status = maki_thompson_rumour_model(ws_graph, gamma=0.01, alpha=alpha, time_limit=20, number_infected=1, all_time_values=True, outputDictQ=False)
        ba_alpha_results.append(ba_sim)
        ws_alpha_results.append(ws_sim)
        ba_alpha_degrees.append(ba_j_degree)
        ws_alpha_degrees.append(ws_j_degree)
        ba__alpha_mean_loads.append(ba_load)
        ws_alpha_mean_loads.append(ws_load)
    ba_results.append(ba_alpha_results)
    ws_results.append(ws_alpha_results)
    ba_degrees.append(ba_alpha_degrees)
    ws_degrees.append(ws_alpha_degrees)
    ba_mean_loads.append(ba__alpha_mean_loads)
    ws_mean_loads.append(ws_alpha_mean_loads)

ValueError: too many values to unpack (expected 3)

In [ ]:
def r_inf_scatter_plot(x,y, title, x_title, y_title):
    fig=go.Figure()